In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

# import your functions
from morphological_operations import dilate, erode, close
# ⚠️ replace `your_module_name` with the filename where your code exists

# ----------------------------
# Load image
# ----------------------------
img = cv2.imread("E:\brand_detection\Brand-Detection\Logos\Adidas\logo1.png", cv2.IMREAD_GRAYSCALE)
if img is None:
    raise FileNotFoundError("Image not found")

# Convert to binary
_, binary = cv2.threshold(img, 127, 255, cv2.THRESH_BINARY)

kernel_size = (5, 5)
iterations = 2
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, kernel_size)

# ----------------------------
# Manual operations
# ----------------------------
manual_dilate = dilate(binary, kernel_size, iterations)
manual_close = close(binary, kernel_size)

# ----------------------------
# OpenCV operations
# ----------------------------
cv_dilate = cv2.dilate(binary, kernel, iterations=iterations)
cv_close = cv2.morphologyEx(binary, cv2.MORPH_CLOSE, kernel)

# ----------------------------
# Difference images
# ----------------------------
dilate_diff = cv2.absdiff(manual_dilate, cv_dilate)
close_diff = cv2.absdiff(manual_close, cv_close)

# ----------------------------
# Display results
# ----------------------------
titles = [
    "Binary Input",
    "Manual Dilation",
    "OpenCV Dilation",
    "Dilation Difference",
    "Manual Close",
    "OpenCV Close",
    "Close Difference"
]

images = [
    binary,
    manual_dilate,
    cv_dilate,
    dilate_diff,
    manual_close,
    cv_close,
    close_diff
]

plt.figure(figsize=(14, 8))
for i, (img, title) in enumerate(zip(images, titles)):
    plt.subplot(2, 4, i + 1)
    plt.imshow(img, cmap="gray")
    plt.title(title)
    plt.axis("off")

plt.tight_layout()
plt.show()

# ----------------------------
# Numeric comparison
# ----------------------------
print("Dilation identical:", np.array_equal(manual_dilate, cv_dilate))
print("Close identical:", np.array_equal(manual_close, cv_close))
print("Dilation pixel difference count:", np.count_nonzero(dilate_diff))
print("Close pixel difference count:", np.count_nonzero(close_diff))
